In [2]:
from utils.read_write import read_csv, to_csv

In [6]:
processedFileString = '_Processed.csv'
people_df=read_csv('../data/People'+processedFileString, 'PERSON_ID')
vehicles_df=read_csv('../data/Vehicles'+processedFileString, 'CRASH_UNIT_ID')
crashes_df=read_csv('../data/Crashes'+processedFileString, 'RD_NO')

In [8]:
tables = {
    'Damage Reimbursement':[
        'Person_ID',
        'Crash_ID',
        'Vehicle_ID',
        'Category',
        'Cost'
    ],
    'Person':[
        'Person_ID',
        'Type',
        'Sex',
        'Age',
        'Row',
        'Safety_Equipment',
        'Airbag_Deployment_Status',
        'Driver_Action',
        'Driver_Vision',
        'Physical_Condition',
        'Injury_Classification',
        'BAC_Result',
        'City',
        'State'
    ],    
    'Vehicle':[
        'Vehicle_ID',
        'Date_ID',
        'Crash_ID',
        'Unit_NO',
        'Unity_Type',
        'Make',
        'Model',
        'License_Plate_State',
        'Year',
        'Defect',
        'Type',
        'Use',
        'Travel_Direction',
        'Maneuver',
        'Occupant_Count'
    ],
    'Datetime':[
        'DateTime_ID',
        'Day',
        'Month',
        'Year',
        'Time'
    ],
    'Crash':[
        'Crash_ID',
        'Crash_Date_ID',
        'Police_Notified_Date_ID',
        'Crash_Location_ID',
        'Crash_Condition_ID',
        'Injury_ID',
        'Primary_Contributory_Cause',
        'Secondary_Contributory_Cause',
        'Number_of_Units',
        'Most_Severe_Injury',
        'Difference_between_crash_date_and_police_notified'
    ],
    'Injury':[
        'Injury_ID',
        'Injuries_Total',
        'Injuries_Fatal',
        'Injuries_Incapacitating',
        'Injuries_Non_Incapacitating',
        'Injuries_No_Indication',
        'Injuries_Reported_Not_Evident',
        'Injuries_Unknown',
    ],
    'Crash Location':[
        'Crash_Location_ID',
        'Street',
        'Street_No',
        'Street_Direction',
        'Beat_Of_Occurrence',
        'Latitude',
        'Longitude',
        'Location'
    ],
    'Crash Condition':[
        'Crash_Condition_ID',
        'Traffic_Control_Device',
        'Traffic_Control_Device_Condition',
        'Weather_Condition',
        'Lighting_Condition',
        'Trafficway_Type',
        'Roadway_Surface_Condition',
        'Road_Defect',
        'Alignment'
    ]
}